In [ ]:
client_region = "norwayeast"
time_to_probe_in_seconds = 360

In [ ]:
account_list = {
    "eastus": [
        "toszrseus001", # eastus ZRS: toszrseus001 - toszrseus020
        "toszrseus002",
        "toszrseus003",
        "toszrseus004",
        "toszrseus005",
        "toszrseus006",
        "toszrseus007",
        "toszrseus008",
        "toszrseus009",
        "toszrseus010",
        "toszrseus011",
        "toszrseus012",
        "toszrseus013",
        "toszrseus014",
        "toszrseus015",
        "toszrseus016",
        "toszrseus017",
        "toszrseus018",
        "toszrseus019",
        "toszrseus020",
    ],
    "eastus2": [
        "toszrseus2001", # eastus2 ZRS: toszrseus2001 - toszrseus2020
        "toszrseus2002",
        "toszrseus2003",
        "toszrseus2004",
        "toszrseus2005",
        "toszrseus2006",
        "toszrseus2007",
        "toszrseus2008",
        "toszrseus2009",
        "toszrseus2010",
        "toszrseus2011",
        "toszrseus2012",
        "toszrseus2013",
        "toszrseus2014",
        "toszrseus2015",
        "toszrseus2016",
        "toszrseus2017",
        "toszrseus2018",
        "toszrseus2019",
        "toszrseus2020", # end of migration related accounts
        "tosazureus001", # eastus2 LRS: tosazureus001 - tosazureus012
        "tosazureus002",
        "tosazureus003",
        "tosazureus004",
        "tosazureus005",
        "tosazureus006",
        "tosazureus007",
        "tosazureus008",
        "tosazureus009",
        "tosazureus010",
        "tosazureus011",
        "tosazureus012",
    ],
    "norwayeast": [
        "toszrsno001",
        "toszrsno002",
        "toszrsno003",
        "toszrsno004",
        "toszrsno005",
        "toszrsno006",
        "toszrsno007",
        "toszrsno008",
        "toszrsno009",
        "toszrsno010",
        "toszrsno011",
        "toszrsno012",
        "toszrsno013",
        "toszrsno014",
        "toszrsno015",
        "toszrsno016",
        "toszrsno017",
        "toszrsno018",
        "toszrsno019",
        "toszrsno020",
    ],
    "southeastasia": [
        "toszrssg01", # ZRS
        "toszrssg02",
        "toszrssg03",
        "toszrssg04",
        "toszrssg05",
        "toszrssg06",
        "toszrssg07",
        "toszrssg08",
        "toszrssg09",
        "toszrssg10",
        "toszrssg11",
        "toszrssg12",
        "toszrssg13",
        "toszrssg14",
        "toszrssg15",
        "toszrssg16",
        "toszrssg17",
        "toszrssg18",
        "toszrssg19",
        "toszrssg20",
        "tosazuresg001", # LRS
        "tosazuresg002",
        "tosazuresg003",
        "tosazuresg004",
        "tosazuresg005",
        "tosazuresg006",
        "tosazuresg007",
        "tosazuresg008",
        "tosazuresg009",
        "tosazuresg010",
        "tosazuresg011",
        "tosazuresg012",
        "tosazuresg013",
        "tosazuresg014",
        "tosazuresg015",
        "tosazuresg016",
        "tosazuresg017",
        "tosazuresg018",
        "tosazuresg019",
        "tosazuresg020",
    ],
    "northeurope": [
        "toszrsie001",
        "toszrsie002",
        "toszrsie003",
        "toszrsie004",
        "toszrsie005",
        "toszrsie006",
        "toszrsie007",
        "toszrsie008",
        "toszrsie009",
        "toszrsie010",
        "toszrsie011",
        "toszrsie012",
        "toszrsie013",
        "toszrsie014",
        "toszrsie015",
        "toszrsie016",
        "toszrsie017",
        "toszrsie018",
        "toszrsie019",
        "toszrsie020",
    ]
}

accounts = account_list[client_region]

# Utils

In [ ]:
import datetime
import json
from xportal.utils import RestHelper, get_endpoint
from xportal.icm import IncidentCreationResult
from urllib.parse import urljoin

# title = f"[ByteDance Connectivity Monitor][Monitor] probe job scheduling failed or running long"
# summary = []

# async def create_incident():
#     now = datetime.datetime.utcnow()
#     body = {
#             "Status": "Active",
#             "Title": title,
#             "Severity": 2,
#             "RoutingId": "mdm://XStore/XInvestigator-Livesite",
#             "CorrelationId": f"BDAccountMonitor{now.strftime('%Y%m%d')}"
#     }
#     if len(summary):
#         body["Summary"] = "<br>".join(summary)
#     result = await RestHelper.fetch_post(
#         urljoin(get_endpoint(), "/api/v1/Icm/CreateIncident"),
#         json.dumps(body, default=str),
#     )
#     return IncidentCreationResult(result)

In [ ]:
# from xportal.utils import in_aks, get_template_path, get_nb_blob_id
# from xportal import kusto
# import datetime

# def get_job_link():
#     job_id = None
#     template_path = None
#     job_link = None
#     job_id = None
#     if in_aks():
#         job_id = get_nb_blob_id().split('_')[0]
#         template_path = get_template_path()
#         job_link = f"https://xportal.trafficmanager.net/xjupyterlitereport?path={job_id}_{template_path}.html"
#     print(job_link)
#     return job_id, job_link

# job_id, job_link = get_job_link()
# if job_link:
#     summary.append(f'''<a href="{job_link}">Monitor Logs</a>''')

In [ ]:
from functools import wraps
import asyncio

def exp_retry(retrycount, initial_backoff=10):
    def decorator(f):
        @wraps(f)
        async def wrapped(*args, **kwargs):
            iteration = 0
            backoff = initial_backoff
            while iteration < retrycount:
                try:
                    return await f(*args, **kwargs)
                except Exception as e:
                    print(f'{f.__name__} failed for the iteration {iteration} with {e}')
                
                iteration += 1
                if iteration < retrycount:
                    backoff *= 2
                    print(f'Waiting for {backoff} seconds then retry...')
                    await asyncio.sleep(backoff)
                    print(f'Retrying for {f.__name__}, iteration: {iteration}')
                else:
                    print(f'Exceeded max retry count for {f.__name__}.')
                    raise Exception(f'Exceeded max retry count for {f.__name__}.')
        return wrapped
    return decorator

# wrap all async operation with retry
@exp_retry(3)
async def with_retry(func, *args, **kwargs):
    return await func(*args, **kwargs)

In [ ]:
from xportal.utils import fetch_post
from typing import List
import json

async def submit_template_job2(template_path: str, parameters: dict=None, outputs: List[str]=None, environment: str=None, destination: str=None) -> str: 
    if environment == None:
        if 'CLOUD_SERVICE_NAME' in os.environ:
            environment = os.environ['CLOUD_SERVICE_NAME']
        else:
            environment = 'Production'

    if template_path.startswith('https://aka.ms/xjpl?templatePath='):
        template_path = template_path.replace("https://aka.ms/xjpl?templatePath=", "")
    if not template_path.startswith('/'):
        template_path = '/' + template_path

    sys_priority = os.getenv("PRIORITY", default="normal")
    # if parameter spcified priority, use it, otherwise use priority inherit from environment
    copy_of_parameters = parameters.copy() if parameters else None
    if copy_of_parameters:
        if copy_of_parameters.get('PRIORITY', None) is None:
            copy_of_parameters['PRIORITY'] = sys_priority
    else:
        copy_of_parameters = {'PRIORITY': sys_priority}

    body = {
        'ProjectName': 'Production',
        'Severity': 'Sev2',
        'Parameters': {
            'InputParametersJson': json.dumps(copy_of_parameters, default=str) if copy_of_parameters else '',
            'Script': template_path,
            'OutputObjectsList': json.dumps(outputs, default=str) if outputs else '',
            "Environment": environment,
            "Destination": destination
        }
    }
    response = await fetch_post('/api/v1/AAIncidentData/RunScriptIncidentData', json.dumps(body, default=str))
    return response['IncidentDiagnosticItemId']

# Scheduling probes

In [ ]:
import asyncio
from xportal import submit_template_job, get_job_response
import time

template = "Xstore/FE/Network/probe.ipynb"

async def submit_job(a, semaphore):
    async with semaphore:
        parameters = {
            "account": a,
            "time_to_probe_in_seconds": time_to_probe_in_seconds,
        }
        
        job_id = await with_retry(submit_template_job2, template, parameters, environment="Dedicated", destination=client_region)
        print(job_id)
        return job_id

async def check_job_status(job_id, semaphore):
    async with semaphore:
        job_response = await with_retry(get_job_response, template, job_id)
        return job_response.status

semaphore = asyncio.Semaphore(4)
tasks = [asyncio.create_task(submit_job(a, semaphore)) for a in accounts]
job_ids = await asyncio.gather(*tasks)

# time.sleep(max(time_to_probe_in_seconds*3, time_to_probe_in_seconds + 300))

# tasks = [asyncio.create_task(check_job_status(job_id, semaphore)) for job_id in job_ids]
# job_status = await asyncio.gather(*tasks)

# probe_failed = False
# for ji, status in enumerate(job_status):
#     print(f'''{job_ids[ji]} : {status}''')
#     if status != "Success":
#         probe_failed = True
#         summary.append(f'''<a href="https://xportal.trafficmanager.net/xjupyterlitereport?path={job_ids[ji]}_/{template}.html">Failed job: {job_ids[ji]}</a>''')

# if probe_failed:
#     await with_retry(create_incident)